In [7]:
#Modules used in this program
import pandas as pd
import pandasql as ps 
import os
from sqlalchemy import create_engine,text


In [8]:
#Adding all the rtc data from csv files to a list 
files = [
 r'output_csv\apsrtc.csv'
,r'output_csv\astc.csv'
,r'output_csv\bihar-state-road-transport-corporation-bsrtc.csv'
,r'output_csv\chandigarh-transport-undertaking-ctu.csv'
,r'output_csv\hrtc.csv'
,r'output_csv\jksrtc.csv'
,r'output_csv\kaac-transport.csv'
,r'output_csv\ksrtc-kerala.csv'
,r'output_csv\ktcl.csv'
,r'output_csv\north-bengal-state-transport-corporation.csv'
,r'output_csv\pepsu.csv'
,r'output_csv\wbtc-ctc.csv'
]




#Reading the CSV file data and creating dataframe list using pandas
dfs = []
for file in files:
    df = pd.read_csv(file)
    df['filename'] = os.path.basename(file).removesuffix('.csv')
    dfs.append(df)

#Concatenating  the dataframe list to single dataframe
df = pd.concat(dfs, ignore_index=True)

#Strip numeric value from seat availability column and update the same in dataframe
df['seat_availability'] = df['seat_availability'].str.replace(r' Seat.*|\n.*','', regex=True).fillna('0')


#Renaming the dataframe columns as required
df_renamed = df.rename(columns={
    'bus_route_name':'route_name',
    'bus_route_link':'route_link',
    'bus_name': 'busname',
    'bus_type':'bustype',
    'start_rating':'star_rating',
    'seat_availability':'seats_available'
    })



In [9]:

#Cleansing FileName as per standard RTC Format
df_renamed = ps.sqldf("""SELECT route_name
         , route_link
         , busname
         , bustype
         , departing_time
         , duration
         , reaching_time
         , star_rating
         , price
         , seats_available
         , case when filename = 'bihar-state-road-transport-corporation-bsrtc' THEN 'bsrtc' 
         when filename = 'chandigarh-transport-undertaking-ctu' THEN 'ctu rtc'
         when filename = 'north-bengal-state-transport-corporation' THEN 'nbstc' 
         when filename = 'wbtc-ctc' then 'wbtc'
         when filename = 'ksrtc-kerala' then 'ksrtc_kerala'
         when filename = 'kaac-transport' then 'kaac'
         else filename 
         end as rtc_name 
         from df_renamed
        WHERE departing_time IS NOT NULL
""")


In [10]:

# Set database credentials.
creds = {'usr': 'root',
            'pwd': 'admin%40123',
            'hst': 'localhost',
            'prt': 3306,
            'dbn': 'redbus'}
# MySQL conection string.
connstr = 'mysql+mysqlconnector://{usr}:{pwd}@{hst}:{prt}/{dbn}'
# Create sqlalchemy engine for MySQL connection.
engine = create_engine(connstr.format(**creds))

In [11]:
#To load the data to bus_routes table in SQL
df_renamed.to_sql(name='bus_routes', con=engine, if_exists='replace', index=False,chunksize=1000)

12391

In [12]:
#To connect MySQL and validate the result from database
with engine.connect() as connection:
    result = connection.execute(text("SELECT * FROM redbus.bus_routes"))
    for row in result:
        print(row)

('Vijayawada to Hyderabad', 'https://www.redbus.in/bus-tickets/vijayawada-to-hyderabad', 'APSRTC - 35105', 'AMARAVATHI (VOLVO / SCANIA A.C Multi Axle)', '04:30', '06h 15m', '10:45', 4.2, 720.0, '26', 'apsrtc')
('Vijayawada to Hyderabad', 'https://www.redbus.in/bus-tickets/vijayawada-to-hyderabad', 'APSRTC - 9362', 'Super Luxury (Non AC Seater 2+2 Push Back)', '04:45', '06h 55m', '11:40', 3.6, 436.0, '13', 'apsrtc')
('Vijayawada to Hyderabad', 'https://www.redbus.in/bus-tickets/vijayawada-to-hyderabad', 'APSRTC - 3565', 'DOLPHIN CRUISE (VOLVO / SCANIA A.C Multi Axle)', '05:30', '05h 55m', '11:25', 4.1, 720.0, '23', 'apsrtc')
('Vijayawada to Hyderabad', 'https://www.redbus.in/bus-tickets/vijayawada-to-hyderabad', 'APSRTC - 3517', 'Super Luxury (Non AC Seater 2+2 Push Back)', '05:30', '06h 40m', '12:10', 4.0, 436.0, '33', 'apsrtc')
('Vijayawada to Hyderabad', 'https://www.redbus.in/bus-tickets/vijayawada-to-hyderabad', 'APSRTC - 3567', 'DOLPHIN CRUISE (VOLVO / SCANIA A.C Multi Axle)', '06